## [In progress]: Dataset with Global Forest Change as a target

In [ ]:
import pathlib
import pyproj
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import skimage.morphology as morph
import torch
import torch.nn.functional as F

from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
from utils import  get_credentials, download_composite

In [ ]:
# Get credentials
# credentials = get_credentials() 

DB_FOLDER = pathlib.Path("/media/tidop/Datos_4TB1/databases")
GFC_FOLDER = DB_FOLDER / "gfc"
GFC_FOLDER.mkdir(parents=True, exist_ok=True)

# As example, we will download the GFC data for 2020 and 2021
lat, lon = -32.56392,-64.72977

download_composite(lat, lon, f"{GFC_FOLDER}/2020.tif", 512, 10, "2020-01-01", "2020-12-31", 0.6)
download_composite(lat, lon, f"{GFC_FOLDER}/2021.tif", 512, 10, "2021-01-01", "2021-12-31", 0.6)

## Bounding box from the lat,lon
## Convert to metric coordinates
tm_crs = query_utm_crs_info(datum_name="WGS84", area_of_interest=AreaOfInterest(lon, lat, lon, lat))
epsg_code = tm_crs[0].code

geo2met = pyproj.Transformer.from_crs('epsg:4326', f'epsg:{epsg_code}', always_xy=True)
x, y = geo2met.transform(lon, lat)

bbox = [x - 2560, y - 2560, x + 2560, y + 2560]

## Turn back to geographic coordinates
met2geo = pyproj.Transformer.from_crs(f'epsg:{epsg_code}', 'epsg:4326', always_xy=True)
lx, ly = met2geo.transform(bbox[0], bbox[1])
rx, ry = met2geo.transform(bbox[2], bbox[3])

bbox_geo = [lx, ly, rx, ry]

raster_file = GFC_FOLDER / "Hansen_GFC-2023-v1.11_lossyear_30S_070W.tif"

## Plot the images
fig, ax = plt.subplots(2, 3, figsize=(12, 8))
with rasterio.open(f"{GFC_FOLDER}/2020.tif") as src1, rasterio.open(f"{GFC_FOLDER}/2021.tif") as src2, \
    rasterio.open(raster_file) as src3:
    # Calculate the window to read
    window = rasterio.windows.from_bounds(*bbox_geo, transform=src3.transform)
    out_image = src3.read(1, window=window)
    
    ax[0,0].imshow(src1.read([3, 2, 1]).transpose(1, 2, 0) * 6/ 10000)
    ax[0,0].set_title("S2 Composite 2020")

    ax[0,1].imshow(src2.read([3, 2, 1]).transpose(1, 2, 0) * 6/ 10000)
    ax[0,1].set_title("S2 Composite 2021")

    out_image_2021 = (out_image == 21).astype(np.uint8)
    ax[0,2].imshow(out_image_2021, cmap='gray')
    ax[0,2].set_title("GFC Ground Truth 2021")

    out_cleaned_2021 = morph.remove_small_objects(out_image_2021, min_size=64, connectivity=1)

    ax[1,0].imshow(out_cleaned_2021, cmap='gray')
    ax[1,0].set_title("GFC Ground Truth 2021 Cleaned") 

    # Resize to shape of 512,512 for GFC 2021 using torch.nn.functional.interpolate
    out_interp_2021 = F.interpolate(torch.tensor(out_cleaned_2021).unsqueeze(0).unsqueeze(0).float(), 
                                     size=(512, 512)).squeeze().numpy()
    # Add 10m resolution when it's exported
    ax[1,1].imshow(out_interp_2021, cmap='gray')
    ax[1,1].set_title("GFC GT 2021 Cleaned Resized")    
    
    # GT Contour
    ax[1,2].contour(out_interp_2021, colors='red', levels=[0.5])
    ax[1,2].imshow(src2.read([3, 2, 1]).transpose(1, 2, 0) * 6 /10000)
    ax[1,2].set_title(f"GT Contour")
        
    plt.show()